## Snowfall Trends Over the Siachen Glacier (2000-2024)

---

### Objective:

Analyze long-term changes in snowfall over the Siachen Glacier using gridded historical weather data from Open-Meteo.

### Method:

- Use a bounding box covering the Siachen Glacier region
- Aggregate snowfall spatially using the mean across grid cells
- Analyze yearly snowfall trends over 25 years

Data source: Open-Mateo Historical Weather API (model=era5)


In [13]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json

In [18]:
# bounding box for Siachen Glacier
LAT_MIN = 35.1667
LAT_MAX = 35.7000
LON_MIN = 76.7667
LON_MAX = 77.4167

START_DATE = "2000-01-01"
END_DATE = "2024-12-31"

RAW_DATA_PATH = Path("../data/raw/siachen_snowfall_daily_raw.json")

In [9]:
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "bounding_box": f"{LAT_MIN},{LON_MIN},{LAT_MAX},{LON_MAX}",
    "start_date": START_DATE,
    "end_date": END_DATE,
    "daily": "snowfall_sum",
    "timezone": "UTC",
    "models": "era5",
}

response = requests.get(url, params=params, timeout=60)
response.raise_for_status()

data = response.json()

In [ ]:
print(response.status_code)
print(type(data), len(data))
cell = data[0]
print(cell.keys())

200
<class 'list'> 6
dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'daily_units', 'daily'])


In [20]:
# saving the raw response
RAW_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

with open(RAW_DATA_PATH, "w") as f:
    json.dump(data, f, indent=2)

print(f"Raw response successfully saved to: {RAW_DATA_PATH}")

Raw response successfully saved to: ../data/raw/siachen_snowfall_daily_raw.json
